# Health and demographic trends by congressional district


This ETL project will create a database that allows for comparison of healthcare and demographic metrics by congressional district, and representation in Congress (including party affiliation).

In [53]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os
import csv
import requests
import json
import geopandas
from os.path import join, basename
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

In [54]:
#Creating SQL engine
engine = create_engine('sqlite:///healthydistricts.db', echo=False)

In [55]:
#Reading in a data set of health outcomes by congressional district downloaded from: 
# https://www.tfah.org/congressional-districts/.
districts_df_org=pd.read_csv('district_data.csv')
districts_df_org.head()

,CD116,State,District,State-District,lackinsurance,lackinsurance_LCL,lackinsurance_UCL,checkup,chekup_LCL,checkup_UCL,...,physical_inactivity_UCL,mcost,mcost_LCL,mcost_UCL,mhlth,mhlth_LCL,mhlth_UCL,obesity,obesity_LCL,obesity_UCL
0,101,AL,1,AL-01,16.3%,16.2%,16.3%,72.3%,72.3%,72.4%,...,33.2%,16.4%,16.3%,16.4%,15.2%,15.1%,#VALUE!,34.4%,34.3%,34.4%
1,102,AL,2,AL-02,16.2%,16.2%,16.3%,72%,72%,72.1%,...,32.7%,16.2%,16.2%,16.3%,15%,14.9%,15%,35.6%,35.5%,35.6%
2,103,AL,3,AL-03,15.7%,15.7%,15.8%,71%,70.9%,71%,...,32.2%,16.1%,16.1%,16.1%,15.3%,15.2%,15.3%,34.4%,34.3%,34.5%
3,104,AL,4,AL-04,15%,15%,15%,70.3%,70.3%,70.4%,...,33.7%,15%,15%,15%,15%,14.9%,15%,34.5%,34.4%,34.5%
4,105,AL,5,AL-05,14.9%,14.9%,14.9%,70%,69.9%,70.1%,...,29.3%,15%,14.9%,15%,14.4%,14.4%,14.5%,34.1%,34%,34.1%


In [56]:
#The LCL and UCL columns represent the lower- and upper- confidence levels for the metrics. 
#Here those columns are removed.
districts_data_reduced = districts_df_org.drop(columns=['lackinsurance_LCL', 
                                                    'lackinsurance_UCL', 
                                                    'chekup_LCL', 
                                                    'checkup_UCL',
                                                    'cholscreen_LCL', 
                                                    'cholscreen_UCL', 
                                                    'diabetes_LCL',
                                                    'diabetes_UCL',
                                                    'flushot_LCL', 
                                                    'flushot_UCL', 
                                                    'ghlth_LCL', 'ghlth_UCL', 
                                                    'physical_inactivity_LCL', 
                                                    'physical_inactivity_UCL',
                                                     'physical_inactivity_LCL', 
                                                    'physical_inactivity_UCL',
                                                    'mcost_LCL', 
                                                    'mcost_UCL',
                                                    'mhlth_LCL', 
                                                    'mhlth_UCL',
                                                    'obesity_LCL', 
                                                    'obesity_UCL',
                                                    'csmoking_LCL', 
                                                    'csmoking_UCL',
                                                   ])


In [57]:
# Renaming C116 to GEOID column for matching with other data 
districts_df=districts_data_reduced.rename(columns={'CD116':"GEOID",
                                                   'State-District':'state_district'
                                                   })
districts_df['GEOID']=districts_df['GEOID'].astype(str).str.zfill(4)

In [58]:
districts_df.head()

,GEOID,State,District,state_district,lackinsurance,checkup,cholscreen,csmoking,diabetes,flushot,ghlth,physical_inactivity,mcost,mhlth,obesity
0,0101,AL,1,AL-01,16.3%,72.3%,81.1%,20%,12.5%,38.8%,22.9%,33.1%,16.4%,15.2%,34.4%
1,0102,AL,2,AL-02,16.2%,72%,81.1%,19.9%,12.7%,38.6%,23.2%,32.6%,16.2%,15%,35.6%
2,0103,AL,3,AL-03,15.7%,71%,80.1%,20.2%,12.5%,39.8%,23.1%,32.1%,16.1%,15.3%,34.4%
3,0104,AL,4,AL-04,15%,70.3%,81.2%,20.2%,12.3%,39.8%,23.5%,33.7%,15%,15%,34.5%
4,0105,AL,5,AL-05,14.9%,70%,81.1%,18.5%,11.2%,39.7%,21.4%,29.2%,15%,14.4%,34.1%


In [59]:
#Converting percentages from strings to floats
districts_df['lackinsurance']=districts_df['lackinsurance'].str.strip('%').astype('float')/100.0
districts_df['checkup']=districts_df['checkup'].str.strip('%').astype('float')/100.0
districts_df['cholscreen']=districts_df['cholscreen'].str.strip('%').astype('float')/100.0
districts_df['csmoking']=districts_df['csmoking'].str.strip('%').astype('float')/100.0
districts_df['diabetes']=districts_df['diabetes'].str.strip('%').astype('float')/100.0
districts_df['flushot']=districts_df['flushot'].str.strip('%').astype('float')/100.0
districts_df['ghlth']=districts_df['ghlth'].str.strip('%').astype('float')/100.0
districts_df['physical_inactivity']=districts_df['physical_inactivity'].str.strip('%').astype('float')/100.0
districts_df['mcost']=districts_df['mcost'].str.strip('%').astype('float')/100.0
districts_df['mhlth']=districts_df['mhlth'].str.strip('%').astype('float')/100.0
districts_df['obesity']=districts_df['obesity'].str.strip('%').astype('float')/100.0

In [60]:
districts_df.head()

,GEOID,State,District,state_district,lackinsurance,checkup,cholscreen,csmoking,diabetes,flushot,ghlth,physical_inactivity,mcost,mhlth,obesity
0,0101,AL,1,AL-01,0.163,0.723,0.811,0.200,0.125,0.388,0.229,0.331,0.164,0.152,0.344
1,0102,AL,2,AL-02,0.162,0.720,0.811,0.199,0.127,0.386,0.232,0.326,0.162,0.150,0.356
2,0103,AL,3,AL-03,0.157,0.710,0.801,0.202,0.125,0.398,0.231,0.321,0.161,0.153,0.344
3,0104,AL,4,AL-04,0.150,0.703,0.812,0.202,0.123,0.398,0.235,0.337,0.150,0.150,0.345
4,0105,AL,5,AL-05,0.149,0.700,0.811,0.185,0.112,0.397,0.214,0.292,0.150,0.144,0.341


In [61]:
acsurl = "https://api.census.gov/data/2018/acs/acs1?get=NAME,B01001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B03003_003E,B29001_001E,B19013_001E,B19301_001E,B25077_001E,B25064_001E,B19083_001E,B25001_001E,B25002_002E,B25003_002E,B25003_003E,B25002_003E&for=congressional%20district:*"

#Requesting API data in JSON format, setting first row imported as headers for dataframe
acs_response = requests.get(f"{acsurl}").json()
acs_df=pd.DataFrame(acs_response)
acs_df.columns = acs_df.iloc[0]
acs_df= acs_df[1:]

In [62]:
acs_df.head()

,NAME,B01001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B03003_003E,...,B25077_001E,B25064_001E,B19083_001E,B25001_001E,B25002_002E,B25003_002E,B25003_003E,B25002_003E,state,congressional district
1,"Congressional District 20 (116th Congress), Ca...",739352,428821,14945,7779,40200,2653,213127,31827,400149,...,647600,1560,0.4690,251904,225394,120933,104461,26510,06,20
2,"Congressional District 13 (116th Congress), Fl...",735935,580519,91631,2073,26641,756,9233,25082,78500,...,208900,1109,0.4805,391150,303069,195680,107389,88081,12,13
3,"Congressional District 1 (116th Congress), Min...",678418,606439,21905,2013,18839,1141,13714,14367,46054,...,177600,775,0.4400,294916,272152,199117,73035,22764,27,01
4,"Congressional District 10 (116th Congress), Ne...",759704,218853,390745,1661,54128,241,75887,18189,160348,...,306400,1170,0.5086,306523,275644,106700,168944,30879,34,10
5,"Congressional District 5 (116th Congress), Ohio",717088,654248,18872,1250,9691,857,10590,21580,37722,...,146400,731,0.4362,313442,289660,211003,78657,23782,39,05


In [63]:
#Converting codes to population types, guide here: http://proximityone.com/cd.htm

acs_df = acs_df.rename(columns={"B01001_001E": "totalpop",
                                      "B02001_002E": "white",
                                      "B02001_003E": "black",
                                      "B02001_004E": "amerindian_native",
                                      "B02001_005E": "asian",
                                      "B02001_006E": "hawaiian_pacific",
                                      "B02001_007E": "other_race",
                                      "B02001_008E": "two_more_races",
                                      "B03003_003E": "hispanic_latino",
                                        "B29001_001E": "citizen_voters",
                                      "B19013_001E": "median_household_income",
                                      "B19301_001E": "per_capita_income",
                                      "B25077_001E": "median_housing_value",
                                      "B25064_001E": "median_gross_rent",
                                      "B19083_001E": "gini_index_of_income_inequality",
                                      "B25001_001E": "total_housing_units",
                                      "B25002_002E": "occupied_units",
                                      "B25003_002E": "owner_occupied_units",
                                      "B25003_003E": "renter_occupied_units",
                                      "B25002_003E": "vacant_units",
                                      "state": "state",
                                      "congressional district": "district"})
#Adding a GEOID Column concatenating State and District Codes to get GEOID, which will be Primary Key in database
acs_df["GEOID"] = acs_df["state"] + acs_df["district"]
acs_df.keys()

Index(['NAME', 'totalpop', 'white', 'black', 'amerindian_native', 'asian',
       'hawaiian_pacific', 'other_race', 'two_more_races', 'hispanic_latino',
       'citizen_voters', 'median_household_income', 'per_capita_income',
       'median_housing_value', 'median_gross_rent',
       'gini_index_of_income_inequality', 'total_housing_units',
       'occupied_units', 'owner_occupied_units', 'renter_occupied_units',
       'vacant_units', 'state', 'district', 'GEOID'],
      dtype='object', name=0)

In [64]:
#Converting to percentages for demographic information
acs_df['totalpop']=acs_df['totalpop'].astype('float')
acs_df['median_household_income']=acs_df['median_household_income'].astype('float')
acs_df['white']=acs_df['white'].astype('float')
acs_df['black']=acs_df['black'].astype('float')
acs_df['asian']=acs_df['asian'].astype('float')
acs_df['amerindian_native']=acs_df['amerindian_native'].astype('float')
acs_df['hawaiian_pacific']=acs_df['hawaiian_pacific'].astype('float')
acs_df['other_race']=acs_df['other_race'].astype('float')
acs_df['two_more_races']=acs_df['two_more_races'].astype('float')
acs_df['hispanic_latino']=acs_df['hispanic_latino'].astype('float')
acs_df['citizen_voters']=acs_df['citizen_voters'].astype('float')
acs_df.head()

,NAME,totalpop,white,black,amerindian_native,asian,hawaiian_pacific,other_race,two_more_races,hispanic_latino,...,median_gross_rent,gini_index_of_income_inequality,total_housing_units,occupied_units,owner_occupied_units,renter_occupied_units,vacant_units,state,district,GEOID
1,"Congressional District 20 (116th Congress), Ca...",739352.0,428821.0,14945.0,7779.0,40200.0,2653.0,213127.0,31827.0,400149.0,...,1560,0.4690,251904,225394,120933,104461,26510,06,20,0620
2,"Congressional District 13 (116th Congress), Fl...",735935.0,580519.0,91631.0,2073.0,26641.0,756.0,9233.0,25082.0,78500.0,...,1109,0.4805,391150,303069,195680,107389,88081,12,13,1213
3,"Congressional District 1 (116th Congress), Min...",678418.0,606439.0,21905.0,2013.0,18839.0,1141.0,13714.0,14367.0,46054.0,...,775,0.4400,294916,272152,199117,73035,22764,27,01,2701
4,"Congressional District 10 (116th Congress), Ne...",759704.0,218853.0,390745.0,1661.0,54128.0,241.0,75887.0,18189.0,160348.0,...,1170,0.5086,306523,275644,106700,168944,30879,34,10,3410
5,"Congressional District 5 (116th Congress), Ohio",717088.0,654248.0,18872.0,1250.0,9691.0,857.0,10590.0,21580.0,37722.0,...,731,0.4362,313442,289660,211003,78657,23782,39,05,3905


In [65]:
acs_df['white'] = acs_df['white']/acs_df['totalpop']
acs_df['black'] = acs_df['black']/acs_df['totalpop']
acs_df['amerindian_native'] = acs_df['amerindian_native']/acs_df['totalpop']
acs_df['asian'] = acs_df['asian']/acs_df['totalpop']
acs_df['hawaiian_pacific'] = acs_df['hawaiian_pacific']/acs_df['totalpop']
acs_df['other_race'] = acs_df['other_race']/acs_df['totalpop']
acs_df['two_more_races'] = acs_df['two_more_races']/acs_df['totalpop']
acs_df['hispanic_latino'] = acs_df['hispanic_latino']/acs_df['totalpop']
acs_df['citizen_voters'] = acs_df['citizen_voters']/acs_df['totalpop']
acs_df.head()

,NAME,totalpop,white,black,amerindian_native,asian,hawaiian_pacific,other_race,two_more_races,hispanic_latino,...,median_gross_rent,gini_index_of_income_inequality,total_housing_units,occupied_units,owner_occupied_units,renter_occupied_units,vacant_units,state,district,GEOID
1,"Congressional District 20 (116th Congress), Ca...",739352.0,0.579996,0.020214,0.010521,0.054372,0.003588,0.288262,0.043047,0.541216,...,1560,0.4690,251904,225394,120933,104461,26510,06,20,0620
2,"Congressional District 13 (116th Congress), Fl...",735935.0,0.788818,0.124510,0.002817,0.036200,0.001027,0.012546,0.034082,0.106667,...,1109,0.4805,391150,303069,195680,107389,88081,12,13,1213
3,"Congressional District 1 (116th Congress), Min...",678418.0,0.893902,0.032288,0.002967,0.027769,0.001682,0.020215,0.021177,0.067884,...,775,0.4400,294916,272152,199117,73035,22764,27,01,2701
4,"Congressional District 10 (116th Congress), Ne...",759704.0,0.288077,0.514338,0.002186,0.071249,0.000317,0.099890,0.023942,0.211066,...,1170,0.5086,306523,275644,106700,168944,30879,34,10,3410
5,"Congressional District 5 (116th Congress), Ohio",717088.0,0.912368,0.026318,0.001743,0.013514,0.001195,0.014768,0.030094,0.052604,...,731,0.4362,313442,289660,211003,78657,23782,39,05,3905


In [66]:
#Requesting current updated congressional data 
#from https://github.com/unitedstates/congress-legislators

congressurl = "https://theunitedstates.io/congress-legislators/legislators-current.csv"
congress_response = requests.get(f"{congressurl}",stream=True).content.decode('utf-8')
cr = csv.reader(congress_response.splitlines(), delimiter=',')
my_list = list(cr)
congress = pd.DataFrame(my_list[1:],columns=my_list[0])
congress['district']=congress['district'].astype(str).str.zfill(2)
#stripping extra spaces from State abbreviations
congress.state=congress.state.str.strip()
congress.head()

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,,,,Sherrod Brown,1952-11-09,M,sen,OH,...,N00003535,S307,"H2OH13033,S6OH00163",5051,400050,27018,Sherrod Brown,,29389,Sherrod Brown
1,Cantwell,Maria,,,,Maria Cantwell,1958-10-13,F,sen,WA,...,N00007836,S275,"S8WA00194,H2WA01054",26137,300018,27122,Maria Cantwell,,39310,Maria Cantwell
2,Cardin,Benjamin,L.,,,Benjamin L. Cardin,1943-10-05,M,sen,MD,...,N00001955,S308,"H6MD03177,S6MD03177",4004,400064,26888,Ben Cardin,,15408,Ben Cardin
3,Carper,Thomas,Richard,,,Thomas R. Carper,1947-01-23,M,sen,DE,...,N00012508,S277,S8DE00079,663,300019,22421,Tom Carper,,15015,Tom Carper
4,Casey,Robert,P.,Jr.,Bob,"Robert P. Casey, Jr.",1960-04-13,M,sen,PA,...,N00027503,S309,S6PA00217,47036,412246,2541,"Bob Casey, Jr.",,40703,Bob Casey Jr.


In [67]:
# Get indexes where type column is sen (Senators)
indexNames = congress[congress['type'] == 'sen'].index
 
# Delete these row indexes from dataFrame
congress.drop(indexNames , inplace=True)
congress.head()

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
28,Aderholt,Robert,B.,,,Robert B. Aderholt,1965-07-22,M,rep,AL,...,N00003028,,H6AL04098,45516,400004,441,Robert B. Aderholt,,29701,Robert Aderholt
30,Bass,Karen,,,,Karen Bass,1953-10-03,F,rep,CA,...,N00031877,,H0CA33117,62502,412404,28963,Karen Bass,,21110,Karen Bass
32,Bilirakis,Gus,M.,,,Gus M. Bilirakis,1963-02-08,M,rep,FL,...,N00027462,,H6FL09070,1022873,412250,17318,Gus M. Bilirakis,,20758,Gus Bilirakis
33,Bishop,Sanford,D.,Jr.,,"Sanford D. Bishop, Jr.",1947-02-04,M,rep,GA,...,N00002674,,H2GA02031,26194,400030,26817,Sanford Bishop Jr.,,29339,Sanford Bishop
35,Blumenauer,Earl,,,,Earl Blumenauer,1948-08-16,M,rep,OR,...,N00007727,,H6OR03064,43809,400033,367,Earl Blumenauer,,29588,Earl Blumenauer


In [68]:
#Getting State FIPS Codes with State Numbers to go with State Abbreviations in Congress file
fips=pd.read_csv('us-state-ansi-fips.csv')
fips=fips.rename(columns={"stname": "name",
                        " st": "number",
                        " stusps": "state"})
fips['number']=fips['number'].astype(str).str.zfill(2)
#Stripping extra spaces from state abbreviations that were preventing a merge
fips.state = fips.state.str.strip()
fips.head()

,name,number,state
0,Alabama,01,AL
1,Alaska,02,AK
2,Arizona,04,AZ
3,Arkansas,05,AR
4,California,06,CA


In [69]:
#Merging on 'state' to get state numbers into congress dataframe results in NaN
congress = congress.merge(fips, on = 'state', how='inner')
#Adding GEOID column
congress["GEOID"] = congress["number"] + congress["district"]

In [70]:
congress.head()

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id,name,number,GEOID
0,Aderholt,Robert,B.,,,Robert B. Aderholt,1965-07-22,M,rep,AL,...,45516,400004,441,Robert B. Aderholt,,29701,Robert Aderholt,Alabama,01,0104
1,Brooks,Mo,,,,Mo Brooks,1954-04-29,M,rep,AL,...,94888,412395,121610,Mo Brooks,,21193,Mo Brooks,Alabama,01,0105
2,Rogers,Mike,,,,Mike Rogers,1958-07-16,M,rep,AL,...,1014740,400341,5705,Mike Rogers (Alabama),,20301,Mike Rogers (Alabama politician),Alabama,01,0103
3,Sewell,Terri,A.,,,Terri Sewell,1965-01-01,F,rep,AL,...,623257,412396,121621,Terri Sewell,,21102,Terri Sewell,Alabama,01,0107
4,Palmer,Gary,J.,,,Gary J. Palmer,1954-05-14,M,rep,AL,...,76094,412608,146274,Gary Palmer,,21500,Gary Palmer (politician),Alabama,01,0106


In [71]:
health_acs_merge= pd.merge(acs_df,districts_df, on='GEOID')
health_acs_merge.keys()

Index(['NAME', 'totalpop', 'white', 'black', 'amerindian_native', 'asian',
       'hawaiian_pacific', 'other_race', 'two_more_races', 'hispanic_latino',
       'citizen_voters', 'median_household_income', 'per_capita_income',
       'median_housing_value', 'median_gross_rent',
       'gini_index_of_income_inequality', 'total_housing_units',
       'occupied_units', 'owner_occupied_units', 'renter_occupied_units',
       'vacant_units', 'state', 'district', 'GEOID', 'State', 'District',
       'state_district', 'lackinsurance', 'checkup', 'cholscreen', 'csmoking',
       'diabetes', 'flushot', 'ghlth', 'physical_inactivity', 'mcost', 'mhlth',
       'obesity'],
      dtype='object')

In [72]:
health_acs_congress_merge=pd.merge(health_acs_merge, congress, on="GEOID")

In [73]:
health_acs_congress_merge.keys()

Index(['NAME', 'totalpop', 'white', 'black', 'amerindian_native', 'asian',
       'hawaiian_pacific', 'other_race', 'two_more_races', 'hispanic_latino',
       'citizen_voters', 'median_household_income', 'per_capita_income',
       'median_housing_value', 'median_gross_rent',
       'gini_index_of_income_inequality', 'total_housing_units',
       'occupied_units', 'owner_occupied_units', 'renter_occupied_units',
       'vacant_units', 'state_x', 'district_x', 'GEOID', 'State', 'District',
       'state_district', 'lackinsurance', 'checkup', 'cholscreen', 'csmoking',
       'diabetes', 'flushot', 'ghlth', 'physical_inactivity', 'mcost', 'mhlth',
       'obesity', 'last_name', 'first_name', 'middle_name', 'suffix',
       'nickname', 'full_name', 'birthday', 'gender', 'type', 'state_y',
       'district_y', 'senate_class', 'party', 'url', 'address', 'phone',
       'contact_form', 'rss_url', 'twitter', 'facebook', 'youtube',
       'youtube_id', 'bioguide_id', 'thomas_id', 'opensecrets_

In [74]:
healthy_districts_df = health_acs_congress_merge[['GEOID',
                         'totalpop',
                         'party',
                         'full_name',
                         'gini_index_of_income_inequality' ,
                         'median_household_income',
                         'white',
                         'black',
                         'amerindian_native',
                         'asian',
                         'hawaiian_pacific', 
                         'other_race', 
                         'two_more_races', 
                         'hispanic_latino',
                         'citizen_voters',
                         'state_district', 
                         'lackinsurance',
                         'csmoking',
                         'diabetes',
                         'obesity',
                         'ghlth',
                         'mhlth',
                         'url',
                         'twitter' 
                        ]]

In [75]:
healthy_districts_df.head()

,GEOID,totalpop,party,full_name,gini_index_of_income_inequality,median_household_income,white,black,amerindian_native,asian,...,citizen_voters,state_district,lackinsurance,csmoking,diabetes,obesity,ghlth,mhlth,url,twitter
0,0620,739352.0,Democrat,Jimmy Panetta,0.4690,75460.0,0.579996,0.020214,0.010521,0.054372,...,0.588203,CA-20,0.158,0.123,0.091,0.240,0.191,0.126,https://panetta.house.gov,RepJimmyPanetta
1,1213,735935.0,Democrat,Charlie Crist,0.4805,52081.0,0.788818,0.124510,0.002817,0.036200,...,0.794756,FL-13,0.187,0.175,0.108,0.277,0.191,0.132,https://crist.house.gov,repcharliecrist
2,2701,678418.0,Republican,Jim Hagedorn,0.4400,61247.0,0.893902,0.032288,0.002967,0.027769,...,0.737495,MN-01,0.098,0.146,0.081,0.310,0.128,0.100,https://hagedorn.house.gov,RepHagedorn
3,3410,759704.0,Democrat,"Donald M. Payne, Jr.",0.5086,59101.0,0.288077,0.514338,0.002186,0.071249,...,0.652953,NJ-10,0.157,0.165,0.124,0.328,0.215,0.137,https://payne.house.gov,RepDonaldPayne
4,3905,717088.0,Republican,Robert E. Latta,0.4362,60500.0,0.912368,0.026318,0.001743,0.013514,...,0.765835,OH-05,0.085,0.182,0.105,0.345,0.162,0.128,https://latta.house.gov,BobLatta


In [76]:
healthy_districts_df.set_index('GEOID')

,totalpop,party,full_name,gini_index_of_income_inequality,median_household_income,white,black,amerindian_native,asian,hawaiian_pacific,...,citizen_voters,state_district,lackinsurance,csmoking,diabetes,obesity,ghlth,mhlth,url,twitter
GEOID,,,,,,,,,,,,,,,,,,,,,
0620,739352.0,Democrat,Jimmy Panetta,0.4690,75460.0,0.579996,0.020214,0.010521,0.054372,0.003588,...,0.588203,CA-20,0.158,0.123,0.091,0.240,0.191,0.126,https://panetta.house.gov,RepJimmyPanetta
1213,735935.0,Democrat,Charlie Crist,0.4805,52081.0,0.788818,0.124510,0.002817,0.036200,0.001027,...,0.794756,FL-13,0.187,0.175,0.108,0.277,0.191,0.132,https://crist.house.gov,repcharliecrist
2701,678418.0,Republican,Jim Hagedorn,0.4400,61247.0,0.893902,0.032288,0.002967,0.027769,0.001682,...,0.737495,MN-01,0.098,0.146,0.081,0.310,0.128,0.100,https://hagedorn.house.gov,RepHagedorn
3410,759704.0,Democrat,"Donald M. Payne, Jr.",0.5086,59101.0,0.288077,0.514338,0.002186,0.071249,0.000317,...,0.652953,NJ-10,0.157,0.165,0.124,0.328,0.215,0.137,https://payne.house.gov,RepDonaldPayne
3905,717088.0,Republican,Robert E. Latta,0.4362,60500.0,0.912368,0.026318,0.001743,0.013514,0.001195,...,0.765835,OH-05,0.085,0.182,0.105,0.345,0.162,0.128,https://latta.house.gov,BobLatta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3618,722226.0,Democrat,Sean Patrick Maloney,0.4665,82731.0,0.761893,0.099362,0.002368,0.029524,0.000590,...,0.726051,NY-18,0.099,0.150,0.085,0.267,0.145,0.116,https://seanmaloney.house.gov,RepSeanMaloney
3911,678001.0,Democrat,Marcia L. Fudge,0.5623,38747.0,0.397482,0.524545,0.002642,0.025795,0.000482,...,0.774045,OH-11,0.136,0.242,0.155,0.379,0.256,0.160,https://fudge.house.gov,RepMarciaFudge
4204,727449.0,Democrat,Madeleine Dean,0.4613,87168.0,0.795039,0.096447,0.000419,0.065983,0.000411,...,0.747438,PA-04,0.066,0.139,0.085,0.253,0.128,0.113,https://dean.house.gov,RepDean


In [77]:
healthy_districts_df=healthy_districts_df.sort_values(by=['state_district'])

In [78]:
healthy_districts_df

,GEOID,totalpop,party,full_name,gini_index_of_income_inequality,median_household_income,white,black,amerindian_native,asian,...,citizen_voters,state_district,lackinsurance,csmoking,diabetes,obesity,ghlth,mhlth,url,twitter
175,0200,737438.0,Republican,Don Young,0.4322,74346.0,0.643519,0.034168,0.151024,0.062895,...,0.721747,AK-00,0.117,0.176,0.074,0.326,0.145,0.113,https://donyoung.house.gov,RepDonYoung
19,0101,715346.0,Republican,Jerry L. Carl,0.4824,46445.0,0.670933,0.275239,0.009774,0.014856,...,0.761120,AL-01,0.163,0.200,0.125,0.344,0.229,0.152,https://carl.house.gov,RepJerryCarl
325,0102,678122.0,Republican,Barry Moore,0.4763,48290.0,0.629199,0.321442,0.004185,0.010974,...,0.761360,AL-02,0.162,0.199,0.127,0.356,0.232,0.150,https://barrymoore.house.gov,RepBarryMoore
229,0103,708409.0,Republican,Mike Rogers,0.4923,45832.0,0.693084,0.259665,0.001835,0.018420,...,0.767712,AL-03,0.157,0.202,0.125,0.344,0.231,0.153,https://mikerogers.house.gov,RepMikeRogersAL
48,0104,686297.0,Republican,Robert B. Aderholt,0.4792,45387.0,0.861266,0.074561,0.006398,0.004476,...,0.751425,AL-04,0.150,0.202,0.123,0.345,0.235,0.150,https://aderholt.house.gov,Robert_Aderholt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,5508,735997.0,Republican,Mike Gallagher,0.4314,61423.0,0.899565,0.016375,0.025837,0.021440,...,0.755716,WI-08,0.082,0.156,0.079,0.321,0.133,0.109,https://gallagher.house.gov,RepGallagher
198,5401,606136.0,Republican,David B. McKinley,0.4695,46867.0,0.939530,0.026959,0.001374,0.009511,...,0.802340,WV-01,0.101,0.228,0.125,0.363,0.224,0.158,https://mckinley.house.gov,RepMcKinley
167,5402,624120.0,Republican,Alexander Mooney,0.4650,48525.0,0.916080,0.046199,0.002166,0.008615,...,0.780632,WV-02,0.096,0.222,0.131,0.377,0.231,0.154,https://mooney.house.gov,RepAlexMooney
309,5403,575576.0,Republican,Carol D. Miller,0.4803,36863.0,0.935664,0.040836,0.000771,0.004026,...,0.795070,WV-03,0.106,0.253,0.141,0.385,0.272,0.171,https://miller.house.gov,RepCarolMiller


In [79]:
c116districts = geopandas.read_file('shapedata/tl_2019_us_cd116.shp')
c116districts['GEOID']=c116districts['GEOID'].astype('str')
c116districts.set_index('GEOID')

,STATEFP,CD116FP,NAMELSAD,LSAD,CDSESSN,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
GEOID,,,,,,,,,,,,
2707,27,07,Congressional District 7,C2,116,G5200,N,86581301397,5058057072,+46.5530202,-095.6759048,"POLYGON ((-97.23909 48.96826, -97.23909 48.968..."
2702,27,02,Congressional District 2,C2,116,G5200,N,6314304732,241279430,+44.4789680,-092.8530418,"POLYGON ((-93.91131 44.54651, -93.91024 44.548..."
2706,27,06,Congressional District 6,C2,116,G5200,N,7460443329,408726474,+45.3368069,-093.8520551,"POLYGON ((-94.64041 45.37072, -94.64039 45.371..."
2708,27,08,Congressional District 8,C2,116,G5200,N,72282135057,12448157466,+47.2503370,-092.9638699,"POLYGON ((-95.18322 47.41277, -95.16598 47.412..."
1701,17,01,Congressional District 1,C2,116,G5200,N,669183735,2067924,+41.5466708,-087.8357472,"POLYGON ((-88.13677 41.42285, -88.13633 41.422..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3613,36,13,Congressional District 13,C2,116,G5200,N,26316820,7203018,+40.8388622,-073.9329821,"POLYGON ((-73.97062 40.81700, -73.97011 40.817..."
3626,36,26,Congressional District 26,C2,116,G5200,N,567624808,35592440,+42.9618255,-078.8409831,"POLYGON ((-79.07537 43.08135, -79.07400 43.083..."
3622,36,22,Congressional District 22,C2,116,G5200,N,13150455532,386871444,+42.8706939,-075.6626641,"POLYGON ((-76.47265 42.00007, -76.47213 42.000..."


In [80]:
healthy_districts_map = pd.merge(healthy_districts_df,c116districts, on='GEOID')

In [82]:
healthy_districts_map=healthy_districts_map.set_index('state_district')

In [84]:
healthy_districts_map=healthy_districts_map.sort_values(by=['state_district'])

In [85]:
healthy_districts_map.head()

,GEOID,totalpop,party,full_name,gini_index_of_income_inequality,median_household_income,white,black,amerindian_native,asian,...,NAMELSAD,LSAD,CDSESSN,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
state_district,,,,,,,,,,,,,,,,,,,,,
AK-00,0200,737438.0,Republican,Don Young,0.4322,74346.0,0.643519,0.034168,0.151024,0.062895,...,Congressional District (at Large),C1,116,G5200,N,1478927050067,245394222619,+63.3473560,-152.8397334,"MULTIPOLYGON (((179.38874 51.94192, 179.40456 ..."
AL-01,0101,715346.0,Republican,Jerry L. Carl,0.4824,46445.0,0.670933,0.275239,0.009774,0.014856,...,Congressional District 1,C2,116,G5200,N,15713378154,2275494808,+31.0018906,-087.7881212,"POLYGON ((-88.46443 31.69795, -88.46418 31.697..."
AL-02,0102,678122.0,Republican,Barry Moore,0.4763,48290.0,0.629199,0.321442,0.004185,0.010974,...,Congressional District 2,C2,116,G5200,N,26270004118,303481339,+31.7020846,-086.0768417,"POLYGON ((-87.42752 31.26274, -87.42750 31.263..."
AL-03,0103,708409.0,Republican,Mike Rogers,0.4923,45832.0,0.693084,0.259665,0.001835,0.018420,...,Congressional District 3,C2,116,G5200,N,19537635148,471179602,+33.1505639,-085.7002277,"POLYGON ((-86.57780 33.76652, -86.57780 33.768..."
AL-04,0104,686297.0,Republican,Robert B. Aderholt,0.4792,45387.0,0.861266,0.074561,0.006398,0.004476,...,Congressional District 4,C2,116,G5200,N,23021992971,573809025,+34.1181796,-087.2138079,"POLYGON ((-88.27459 33.53425, -88.27450 33.534..."


In [86]:
# #Exporting dataframe to SQLite
# healthy_districts_map.to_sql('healthy_districts_map', con=engine,if_exists='replace')

In [87]:
healthy_districts_df.head()

,GEOID,totalpop,party,full_name,gini_index_of_income_inequality,median_household_income,white,black,amerindian_native,asian,...,citizen_voters,state_district,lackinsurance,csmoking,diabetes,obesity,ghlth,mhlth,url,twitter
175,0200,737438.0,Republican,Don Young,0.4322,74346.0,0.643519,0.034168,0.151024,0.062895,...,0.721747,AK-00,0.117,0.176,0.074,0.326,0.145,0.113,https://donyoung.house.gov,RepDonYoung
19,0101,715346.0,Republican,Jerry L. Carl,0.4824,46445.0,0.670933,0.275239,0.009774,0.014856,...,0.761120,AL-01,0.163,0.200,0.125,0.344,0.229,0.152,https://carl.house.gov,RepJerryCarl
325,0102,678122.0,Republican,Barry Moore,0.4763,48290.0,0.629199,0.321442,0.004185,0.010974,...,0.761360,AL-02,0.162,0.199,0.127,0.356,0.232,0.150,https://barrymoore.house.gov,RepBarryMoore
229,0103,708409.0,Republican,Mike Rogers,0.4923,45832.0,0.693084,0.259665,0.001835,0.018420,...,0.767712,AL-03,0.157,0.202,0.125,0.344,0.231,0.153,https://mikerogers.house.gov,RepMikeRogersAL
48,0104,686297.0,Republican,Robert B. Aderholt,0.4792,45387.0,0.861266,0.074561,0.006398,0.004476,...,0.751425,AL-04,0.150,0.202,0.123,0.345,0.235,0.150,https://aderholt.house.gov,Robert_Aderholt


In [88]:
#Exporting dataframe to SQLite
healthy_districts_df.to_sql('healthy_districts', con=engine,if_exists='replace')

In [111]:
table_data_df=healthy_districts_df[['state_district',
                             'full_name',
                             'party',
                             'median_household_income',
                             'amerindian_native',
                             'asian',
                             'black',
                             'white',
                             'hispanic_latino',
                             'lackinsurance',
                             'csmoking',
                             'diabetes',
                             'obesity']]

table_data_df.head()

,state_district,full_name,party,median_household_income,amerindian_native,asian,black,white,hispanic_latino,lackinsurance,csmoking,diabetes,obesity
175,AK-00,Don Young,Republican,74346.0,0.151024,0.062895,0.034168,0.643519,0.071964,0.117,0.176,0.074,0.326
19,AL-01,Jerry L. Carl,Republican,46445.0,0.009774,0.014856,0.275239,0.670933,0.033747,0.163,0.200,0.125,0.344
325,AL-02,Barry Moore,Republican,48290.0,0.004185,0.010974,0.321442,0.629199,0.036797,0.162,0.199,0.127,0.356
229,AL-03,Mike Rogers,Republican,45832.0,0.001835,0.018420,0.259665,0.693084,0.031199,0.157,0.202,0.125,0.344
48,AL-04,Robert B. Aderholt,Republican,45387.0,0.006398,0.004476,0.074561,0.861266,0.065177,0.150,0.202,0.123,0.345


In [113]:
table_data_df= table_data_df.rename(columns = {'state_district': 'District',
                                     'full_name': 'Representative',
                                     'party': 'Political Party',
                                     'median_household_income': 'Median Income',
                                     'amerindian_native': 'American Indian/Alaskan Native',
                                     'asian': 'Asian',
                                    'black': 'Black',
                                     'white': 'White',
                                     'hispanic_latino': "Hispanic/Latino All Races",
                                     'lackinsurance': 'Uninsured',
                                     'csmoking': 'Currently Smoking',
                                     'diabetes': 'Adult Diabetics',
                                     'obesity': 'Obesity'},
                                   
                                   
                                   
                                   
                                   
                                   )



In [114]:
table_data_df

,District,Representative,Political Party,Median Income,American Indian/Alaskan Native,Asian,Black,White,Hispanic/Latino All Races,Uninsured,Currently Smoking,Adult Diabetics,Obesity
175,AK-00,Don Young,Republican,74346.0,0.151024,0.062895,0.034168,0.643519,0.071964,0.117,0.176,0.074,0.326
19,AL-01,Jerry L. Carl,Republican,46445.0,0.009774,0.014856,0.275239,0.670933,0.033747,0.163,0.200,0.125,0.344
325,AL-02,Barry Moore,Republican,48290.0,0.004185,0.010974,0.321442,0.629199,0.036797,0.162,0.199,0.127,0.356
229,AL-03,Mike Rogers,Republican,45832.0,0.001835,0.018420,0.259665,0.693084,0.031199,0.157,0.202,0.125,0.344
48,AL-04,Robert B. Aderholt,Republican,45387.0,0.006398,0.004476,0.074561,0.861266,0.065177,0.150,0.202,0.123,0.345
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,WI-08,Mike Gallagher,Republican,61423.0,0.025837,0.021440,0.016375,0.899565,0.054284,0.082,0.156,0.079,0.321
198,WV-01,David B. McKinley,Republican,46867.0,0.001374,0.009511,0.026959,0.939530,0.012642,0.101,0.228,0.125,0.363
167,WV-02,Alexander Mooney,Republican,48525.0,0.002166,0.008615,0.046199,0.916080,0.021675,0.096,0.222,0.131,0.377
309,WV-03,Carol D. Miller,Republican,36863.0,0.000771,0.004026,0.040836,0.935664,0.007606,0.106,0.253,0.141,0.385


In [118]:
table_data_df.style.format({'Median Income':"${:,.0f}",
                             'American Indian/Alaskan Native':"{:.2%}",
                             'Asian':"{:.2%}",
                             'Black':"{:.2%}",
                             'White':"{:.2%}",
                             'Hispanic/Latino All Races':"{:.2%}",
                             'Uninsured':"{:.2%}",
                             'Currently Smoking':"{:.2%}",
                             'Adult Diabetics':"{:.2%}",
                             'Obesity':"{:.2%}"
                           
                           
                           
                           })

,District,Representative,Political Party,Median Income,American Indian/Alaskan Native,Asian,Black,White,Hispanic/Latino All Races,Uninsured,Currently Smoking,Adult Diabetics,Obesity
175,AK-00,Don Young,Republican,"$74,346",15.10%,6.29%,3.42%,64.35%,7.20%,11.70%,17.60%,7.40%,32.60%
19,AL-01,Jerry L. Carl,Republican,"$46,445",0.98%,1.49%,27.52%,67.09%,3.37%,16.30%,20.00%,12.50%,34.40%
325,AL-02,Barry Moore,Republican,"$48,290",0.42%,1.10%,32.14%,62.92%,3.68%,16.20%,19.90%,12.70%,35.60%
229,AL-03,Mike Rogers,Republican,"$45,832",0.18%,1.84%,25.97%,69.31%,3.12%,15.70%,20.20%,12.50%,34.40%
48,AL-04,Robert B. Aderholt,Republican,"$45,387",0.64%,0.45%,7.46%,86.13%,6.52%,15.00%,20.20%,12.30%,34.50%
55,AL-05,Mo Brooks,Republican,"$57,174",0.43%,1.68%,18.04%,74.40%,5.34%,14.90%,18.50%,11.20%,34.10%
238,AL-06,Gary J. Palmer,Republican,"$66,835",0.21%,1.92%,15.31%,78.01%,4.70%,13.50%,16.50%,10.00%,31.60%
376,AL-07,Terri Sewell,Democrat,"$37,123",0.29%,0.76%,63.12%,33.54%,3.53%,20.40%,22.00%,15.60%,40.70%
64,AR-01,"Eric A. ""Rick"" Crawford",Republican,"$41,553",0.35%,0.39%,16.92%,77.52%,3.42%,11.30%,21.30%,12.30%,35.70%
139,AR-02,J. Hill,Republican,"$50,381",0.53%,1.70%,22.85%,70.65%,5.29%,11.00%,19.20%,10.20%,31.90%


In [119]:
table_data_df.set_index('District').sort_values(by=['District'])

,Representative,Political Party,Median Income,American Indian/Alaskan Native,Asian,Black,White,Hispanic/Latino All Races,Uninsured,Currently Smoking,Adult Diabetics,Obesity
District,,,,,,,,,,,,
AK-00,Don Young,Republican,74346.0,0.151024,0.062895,0.034168,0.643519,0.071964,0.117,0.176,0.074,0.326
AL-01,Jerry L. Carl,Republican,46445.0,0.009774,0.014856,0.275239,0.670933,0.033747,0.163,0.200,0.125,0.344
AL-02,Barry Moore,Republican,48290.0,0.004185,0.010974,0.321442,0.629199,0.036797,0.162,0.199,0.127,0.356
AL-03,Mike Rogers,Republican,45832.0,0.001835,0.018420,0.259665,0.693084,0.031199,0.157,0.202,0.125,0.344
AL-04,Robert B. Aderholt,Republican,45387.0,0.006398,0.004476,0.074561,0.861266,0.065177,0.150,0.202,0.123,0.345
...,...,...,...,...,...,...,...,...,...,...,...,...
WI-08,Mike Gallagher,Republican,61423.0,0.025837,0.021440,0.016375,0.899565,0.054284,0.082,0.156,0.079,0.321
WV-01,David B. McKinley,Republican,46867.0,0.001374,0.009511,0.026959,0.939530,0.012642,0.101,0.228,0.125,0.363
WV-02,Alexander Mooney,Republican,48525.0,0.002166,0.008615,0.046199,0.916080,0.021675,0.096,0.222,0.131,0.377


In [120]:
#Exporting dataframe to SQLite
table_data_df.to_html('Templates/data.html', index=False, classes=['table', 'table-striped', 'table-hover'])

In [79]:
table_data_df.to_html

<bound method DataFrame.to_html of     District        Representative Political Party Median Income  \
0      CA-20         Jimmy Panetta        Democrat         75460   
1      FL-13         Charlie Crist        Democrat         52081   
2      MN-01          Jim Hagedorn      Republican         61247   
3      NJ-10  Donald M. Payne, Jr.        Democrat         59101   
4      OH-05       Robert E. Latta      Republican         60500   
..       ...                   ...             ...           ...   
441    NY-18  Sean Patrick Maloney        Democrat         82731   
442    OH-11       Marcia L. Fudge        Democrat         38747   
443    PA-04        Madeleine Dean        Democrat         87168   
444    TX-25        Roger Williams      Republican         71031   
445    VA-09    H. Morgan Griffith      Republican         41520   

     American Indian/Alaskan Native     Asian     Black     White  \
0                          0.010521  0.054372  0.020214  0.579996   
1         